# 5회차 실습 과제 — 정답

## 문제 항목

| 문제     | 주제                                       | 핵심 학습 포인트                              |
|----------|--------------------------------------------|-----------------------------------------------|
| 문제 1   | 3개 공장의 배터리 수명 비교 (ANOVA)        | 가정 검정 → 검정 선택 → 사후검정 → 효과크기   |
| 문제 2   | 직원 데이터 상관분석                       | Pearson vs Spearman, 산점도, 상관 vs 인과      |
| 문제 3   | 앱 UI 변경 A/B 테스트 (전환율)             | 실험 설계 → 표본 크기 산정 → 검정 → 의사결정   |
| 문제 4   | 학습 앱 A/B 테스트 (연속형 지표)           | 연속형 A/B 테스트, CLT, 모수 vs 비모수         |

In [ ]:
# 필수 라이브러리 Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from itertools import combinations
import pingouin as pg
import scikit_posthocs as sp
from statsmodels.stats.proportion import proportions_ztest, proportion_effectsize, proportion_confint, confint_proportions_2indep
from statsmodels.stats.power import NormalIndPower
from scipy.stats import skew
import seaborn as sns
import warnings
import platform

warnings.filterwarnings('ignore')

# 운영체제별 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':
    plt.rcParams['font.family'] = 'AppleGothic'
else:
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

print("=" * 60)
print("5회차 실습 과제 — 정답")
print("=" * 60)

---

## 문제 1: 3개 공장의 배터리 수명 비교 (ANOVA)

전자제품 회사에서 3개 공장(A, B, C)에서 생산한 배터리의 수명(시간)이 동일한지 검증합니다.
각 공장에서 20개씩 배터리를 무작위로 추출하여 수명을 측정했습니다.

**분석 목표**: 3개 공장에서 생산한 배터리의 평균 수명에 통계적으로 유의한 차이가 있는지 검정합니다.

**주어진 데이터:**

In [ ]:
print("\n[문제 1] 3개 공장의 배터리 수명 비교")
print("=" * 50)

np.random.seed(691)
factory_a = np.round(np.random.normal(loc=480, scale=30, size=20), 1)   # A공장: 평균 480시간
factory_b = np.round(np.random.normal(loc=510, scale=25, size=20), 1)   # B공장: 평균 510시간
factory_c = np.round(np.random.normal(loc=495, scale=35, size=20), 1)   # C공장: 평균 495시간

for name, data in [('A공장', factory_a), ('B공장', factory_b), ('C공장', factory_c)]:
    print(f"  {name} (n={len(data)}): 평균={data.mean():.1f}시간, SD={data.std(ddof=1):.1f}시간")

In [ ]:
# 분포 시각화: 박스플롯 + 히스토그램
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

colors = ['#3B82F6', '#F59E0B', '#10B981']
bp = axes[0].boxplot([factory_a, factory_b, factory_c],
                      labels=['A공장', 'B공장', 'C공장'],
                      patch_artist=True, widths=0.5)
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)

axes[0].set_ylabel('배터리 수명 (시간)', fontsize=12)
axes[0].set_title('공장별 배터리 수명 분포', fontsize=13, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

for data, name, color in [(factory_a, 'A공장', '#3B82F6'),
                           (factory_b, 'B공장', '#F59E0B'),
                           (factory_c, 'C공장', '#10B981')]:
    axes[1].hist(data, bins=10, alpha=0.4, color=color, edgecolor='white', label=name)
    axes[1].axvline(data.mean(), color=color, linestyle='--', alpha=0.8)

axes[1].set_xlabel('배터리 수명 (시간)', fontsize=12)
axes[1].set_ylabel('빈도', fontsize=12)
axes[1].set_title('수명 분포 비교', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 문제 1-1: 가설 설정 및 가정 검정

**(a)** 분석 목표에 맞는 **귀무가설(H₀)** 과 **대립가설(H₁)** 을 수식으로 설정하세요.

**(b)** 각 공장 데이터의 **정규성 검정**(Shapiro-Wilk)을 수행하세요.

**(c)** 3개 공장의 **Q-Q Plot**을 나란히 그리세요.

**(d)** 정규성이 충족된다면, **등분산 검정**(Levene)을 수행하세요.

**(e)** 가정 검정 결과를 종합하여 **어떤 검정을 사용할지** 결정하세요.
의사결정 과정을 단계별로 설명하세요.

> **의사결정 흐름**: 정규성 확인 → (충족 시) 등분산 확인 → 검정 방법 선택

In [ ]:
print("\n[문제 1-1] 가설 설정 및 가정 검정")
print("-" * 40)

# (a) 가설 설정
print("(a) 가설 설정:")
print(f"    H₀: μ_A = μ_B = μ_C (3개 공장의 평균 배터리 수명이 모두 같습니다)")
print(f"    H₁: 적어도 하나의 공장 평균 수명이 다릅니다")

# (b) 정규성 검정
normality_ok = True

print(f"\n(b) Shapiro-Wilk 정규성 검정:")
for name, data in [('A공장', factory_a), ('B공장', factory_b), ('C공장', factory_c)]:
    stat_sw, p_sw = stats.shapiro(data)
    result = "정규성 유지" if p_sw > 0.05 else "정규성 위반"
    if p_sw <= 0.05:
        normality_ok = False
    print(f"    {name}: W={stat_sw:.4f}, p={p_sw:.4f} → {result}")

print(f"    종합: {'모든 집단 정규성 충족' if normality_ok else '정규성 위반 집단 있음'}")

In [ ]:
# (c) Q-Q Plot
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle("정규성 확인 — Q-Q Plot", fontsize=14, fontweight='bold')

for ax, data, name, color in [(axes[0], factory_a, 'A공장', '#3B82F6'),
                                (axes[1], factory_b, 'B공장', '#F59E0B'),
                                (axes[2], factory_c, 'C공장', '#10B981')]:
    stats.probplot(data, dist="norm", plot=ax)
    sw_p = stats.shapiro(data).pvalue
    ax.set_title(f"{name} Q-Q Plot (p={sw_p:.4f})", fontsize=12, fontweight="bold")
    ax.get_lines()[0].set_markerfacecolor(color)
    ax.get_lines()[0].set_markeredgecolor(color)
    ax.get_lines()[0].set_markersize(5)
    ax.get_lines()[1].set_color('red')
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# (d) 등분산 검정
lev_stat, lev_p = stats.levene(factory_a, factory_b, factory_c)

print(f"(d) Levene 등분산 검정:")
print(f"    F={lev_stat:.4f}, p={lev_p:.4f}")
equal_var = lev_p > 0.05
verdict_lev = "등분산 가정 충족" if equal_var else "등분산 가정 위반"
print(f"    판정: {verdict_lev}")

# (e) 검정 방법 결정
print(f"\n(e) 검정 방법 결정:")
print(f"    [1] 정규성 — {'모든 집단 정규성 충족' if normality_ok else '정규성 위반 집단 있음'}")
print(f"    [2] 등분산 — p={lev_p:.4f} → {verdict_lev}")
if normality_ok and equal_var:
    test_method = "One-way ANOVA"
elif normality_ok and not equal_var:
    test_method = "Welch's ANOVA"
else:
    test_method = "Kruskal-Wallis"
print(f"    [3] 결론 → 사용할 검정: {test_method}")

### 문제 1-2: ANOVA 검정 수행 및 효과크기

**(a)** 1-1에서 결정한 검정을 수행하세요 (α = 0.05).

**(b)** 전체 효과크기(η² 또는 η²_H)를 계산하고 해석하세요.

**(c)** ω²(오메가제곱)을 계산할 수 있다면 함께 보고하세요.

**(d)** 검정 결과(p-value)와 효과크기를 종합하여 결론을 내리세요.

In [ ]:
print("\n[문제 1-2] ANOVA 검정 수행 및 효과크기")
print("-" * 40)

# 표본 데이터 생성
groups = ['A공장'] * 20 + ['B공장'] * 20 + ['C공장'] * 20
lifetimes = np.concatenate([factory_a, factory_b, factory_c])
df_battery = pd.DataFrame({'공장': groups, '수명': lifetimes})

print("배터리 데이터 샘플:")
display(df_battery.sample(5))
print("기술통계:")
display(df_battery.describe().round(2))

In [ ]:
# (a) One-way ANOVA
f_stat, f_p = stats.f_oneway(factory_a, factory_b, factory_c)

print(f"(a) One-way ANOVA 검정 결과:")
print(f"    F = {f_stat:.4f}")
print(f"    p-value = {f_p:.6f}")
verdict_1 = "H₀ 기각 → 적어도 하나의 공장 평균이 다릅니다" if f_p <= 0.05 else "H₀ 기각 실패"
print(f"    판정 (α=0.05): {verdict_1}")

aov = pg.anova(dv='수명', between='공장', data=df_battery, detailed=True)
display(aov)

# (b) 전체 효과크기 (η²)
eta_sq = aov['np2'].values[0]
size_eta = '작은' if eta_sq < 0.06 else '중간' if eta_sq < 0.14 else '큰'

print(f"\n(b) 전체 효과크기:")
print(f"    η² = {eta_sq:.4f}")
print(f"    해석 기준: 0.01 작은, 0.06 중간, 0.14 큰")
print(f"    판단: {size_eta} 효과 크기입니다")

# (c) ω²
ss_b = aov['SS'].values[0]
ss_w = aov['SS'].values[1]
df_b = aov['DF'].values[0]
ms_w = aov['MS'].values[1]
ss_t = ss_b + ss_w

omega_sq = (ss_b - df_b * ms_w) / (ss_t + ms_w)

print(f"\n(c) ω² = {omega_sq:.4f}")
print(f"    η²({eta_sq:.4f})보다 보수적인 추정치입니다")

# (d) 종합 결론
print(f"\n(d) 종합 결론:")
print(f"    3개 공장의 배터리 수명에 통계적으로 유의한 차이가 있습니다")
print(f"    (One-way ANOVA, F={f_stat:.2f}, p={f_p:.6f})")
print(f"    효과크기 η²={eta_sq:.4f}({size_eta})로, 공장 차이가 전체 변동의 {eta_sq*100:.1f}%를 설명합니다")

### 문제 1-3: 사후검정 — 어떤 공장 쌍이 다른가?

ANOVA가 유의하다면, **"적어도 하나가 다르다"** 는 것만 알 수 있습니다.
**어떤 공장 쌍**이 다른지 확인하려면 사후검정이 필요합니다.

**(a)** 사용한 검정 방법에 맞는 사후검정을 수행하세요.

> | 검정 방법 | 사후검정 |
> |-----------|---------|
> | One-way ANOVA | Tukey HSD |
> | Welch's ANOVA | Games-Howell |
> | Kruskal-Wallis | Dunn 검정 (Bonferroni 보정) |

**(b)** 쌍별 효과크기를 계산하세요.

> - Tukey HSD 후 → Cohen's d (`pg.compute_effsize(x, y, eftype='cohen')`)
> - Games-Howell 후 → Hedges' g (`pg.compute_effsize(x, y, eftype='hedges')`)
> - Dunn 검정 후 → 중앙값 차이로 방향과 크기 해석

**(c)** 사후검정 결과를 종합하여, 구체적으로 어떤 공장이 어떻게 다른지 해석하세요.

**(d)** 공장별 평균 배터리 수명을 막대 그래프(±SEM 오차막대 포함)로 시각화하세요.

In [ ]:
print("\n[문제 1-3] 사후검정")
print("-" * 40)

# (a) Tukey HSD
print(f"(a) Tukey HSD 사후검정 결과:")
tukey = pairwise_tukeyhsd(df_battery['수명'], df_battery['공장'], alpha=0.05)
print(tukey)

# (b) 쌍별 효과크기
print(f"\n(b) 쌍별 효과크기 (Cohen's d):")
group_data = {'A공장': factory_a, 'B공장': factory_b, 'C공장': factory_c}
for (g1, d1), (g2, d2) in combinations(group_data.items(), 2):
    d = pg.compute_effsize(d1, d2, eftype='cohen')
    size_d = '매우 작은' if abs(d) < 0.2 else '작은' if abs(d) < 0.5 else '중간' if abs(d) < 0.8 else '큰'
    print(f"    {g1} vs {g2}: d = {d:.3f} ({size_d} 효과)")

# (c) 종합 해석
print(f"\n(c) 종합 해석:")
print(f"    Tukey HSD 결과, 모든 공장 쌍에서 유의한 차이를 보입니다.")
print(f"    B공장의 평균 수명이 가장 길고, A공장이 가장 짧습니다.")
print(f"    특히 A공장 vs B공장의 Cohen's d가 가장 크므로 실질적 차이도 큽니다.")

In [ ]:
# (d) 막대 그래프 (±SEM)
fig, ax = plt.subplots(figsize=(10, 6))

means = [d.mean() for d in group_data.values()]
sems = [d.std(ddof=1) / np.sqrt(len(d)) for d in group_data.values()]

bars = ax.bar(group_data.keys(), means, yerr=sems, capsize=5,
              color=colors, alpha=0.7, edgecolor='white', linewidth=1.5)
for bar, mean, sem in zip(bars, means, sems):
    ax.text(bar.get_x() + bar.get_width()/2., mean + sem + 1,
            f'{mean:.1f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

ax.set_ylabel('배터리 수명 (시간)', fontsize=12)
ax.set_title('공장별 평균 배터리 수명 (±SEM)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

---

## 문제 2: 직원 데이터 상관분석

한 IT 기업에서 직원 30명의 데이터를 수집했습니다.
경력(년), 월급여(만원), 프로젝트 완료 수, 직무 만족도(1~10점)의 관계를 분석합니다.

**분석 목표**:
- 변수 간 상관관계를 파악하고, 어떤 변수 쌍이 유의한 상관을 보이는지 확인합니다.
- Pearson과 Spearman 상관계수를 비교하여 데이터의 특성을 파악합니다.

**주어진 데이터:**

In [ ]:
print("\n[문제 2] 직원 데이터 상관분석")
print("=" * 50)

np.random.seed(552)
n_emp = 30

experience = np.round(np.random.uniform(1, 15, n_emp), 1)
salary = np.round(2500 + 180 * experience + np.random.normal(0, 400, n_emp), 0)
projects = np.round(np.clip(2 + 0.8 * experience + np.random.normal(0, 3, n_emp), 0, 30), 0).astype(int)
satisfaction = np.round(np.clip(4 + 0.3 * experience - 0.0005 * salary + np.random.normal(0, 1.8, n_emp), 1, 10), 1)

df_emp = pd.DataFrame({
    '경력(년)': experience,
    '월급여(만원)': salary,
    '프로젝트수': projects,
    '만족도': satisfaction
})
print("직원 데이터 샘플:")
display(df_emp.sample(5))
print("기술통계:")
display(df_emp.describe().round(2))

### 문제 2-1: 산점도와 상관 행렬

**(a)** `경력(년)`과 `월급여(만원)`의 산점도를 그리고, 회귀선을 추가하세요.
시각적으로 어떤 관계가 보이는지 서술하세요.

**(b)** 전체 변수의 **Pearson 상관 행렬**을 계산하고 히트맵으로 시각화하세요.

> `df_emp.corr(method='pearson')` + `seaborn.heatmap()`

**(c)** 전체 변수의 **Spearman 상관 행렬**도 함께 히트맵으로 그려서 나란히 비교하세요.

In [ ]:
print("\n[문제 2-1] 산점도와 상관 행렬")
print("-" * 40)

# (a) 경력 vs 월급여 산점도 + 회귀선
fig, ax = plt.subplots(figsize=(10, 7))

ax.scatter(experience, salary, color='#6366F1', alpha=0.7, s=60, edgecolors='white')

z = np.polyfit(experience, salary, 1)
p_line = np.poly1d(z)
x_line = np.linspace(experience.min(), experience.max(), 100)
ax.plot(x_line, p_line(x_line), color='#EF4444', linewidth=2, linestyle='--',
        label=f'회귀선 (기울기={z[0]:.1f})')

ax.set_xlabel('경력 (년)', fontsize=12)
ax.set_ylabel('월급여 (만원)', fontsize=12)
ax.set_title('경력 vs 월급여', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("(a) 시각적 관찰 결과 해석:")
print(f"    경력이 늘어날수록 월급여가 증가하는 양의 선형 관계가 관찰됩니다.")
print(f"    일부 이상치가 있으나 전반적으로 선형 패턴이 뚜렷합니다.")

In [ ]:
# Pearson vs Spearman 상관 히트맵
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# (b) Pearson 상관 행렬 히트맵
corr_pearson = df_emp.corr(method='pearson')
sns.heatmap(corr_pearson, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=0.5, ax=axes[0])
axes[0].set_title('Pearson 상관 행렬', fontsize=13, fontweight='bold')

# (c) Spearman 상관 행렬 히트맵
corr_spearman = df_emp.corr(method='spearman')
sns.heatmap(corr_spearman, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=0.5, ax=axes[1])
axes[1].set_title('Spearman 상관 행렬', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

### 문제 2-2: 상관계수 가설검정

**(a)** `경력(년)`과 `월급여(만원)` 사이의 **Pearson 상관계수**와 **p-value**를 구하세요.

**(b)** 같은 변수 쌍에 대해 **Spearman 상관계수**와 **p-value**를 구하세요.

**(c)** 두 상관계수를 비교하세요. 차이가 크다면/작다면 어떤 의미인지 설명하세요.

In [ ]:
print("\n[문제 2-2] 상관계수 가설검정")
print("-" * 40)

# (a) Pearson 상관계수
r_pearson, p_pearson = stats.pearsonr(experience, salary)

print(f"(a) Pearson 상관계수:")
print(f"    r = {r_pearson:.4f}")
print(f"    p = {p_pearson:.4f}")
r_strength = '약한' if abs(r_pearson) < 0.3 else '중간' if abs(r_pearson) < 0.7 else '강한'
print(f"    강도: {r_strength} 양의 상관")
verdict_p = "유의한 상관" if p_pearson < 0.05 else "유의하지 않은 상관"
print(f"    판정: {verdict_p} (p={'<' if p_pearson < 0.05 else '>'} 0.05)")

# (b) Spearman 상관계수
r_spearman, p_spearman = stats.spearmanr(experience, salary)

print(f"\n(b) Spearman 상관계수:")
print(f"    ρ = {r_spearman:.4f}")
print(f"    p = {p_spearman:.4f}")
verdict_s = "유의한 상관" if p_spearman < 0.05 else "유의하지 않은 상관"
print(f"    판정: {verdict_s}")

# (c) 비교
print(f"\n(c) Pearson vs Spearman 비교:")
print(f"    Pearson r  = {r_pearson:.4f}")
print(f"    Spearman ρ = {r_spearman:.4f}")
diff_corr = abs(r_pearson - r_spearman)
print(f"    차이: {diff_corr:.4f}")
if diff_corr < 0.1:
    print(f"    해석: 두 값이 비슷하므로 경력과 월급여의 관계가 선형에 가깝습니다.")
elif r_spearman > r_pearson:
    print(f"    해석: Spearman이 더 높으므로 비선형이지만 단조적인 관계가 있습니다.")
else:
    print(f"    해석: Pearson이 더 높으므로 이상치가 Pearson을 부풀렸을 가능성이 있습니다.")

r_squared = r_pearson ** 2
print(f"\n    결정계수 r² = {r_squared:.4f}")
print(f"    해석: 경력이 월급여 분산의 {r_squared*100:.1f}%를 설명합니다.")

### 문제 2-3: 다변량 상관분석과 해석

**(a)** `pg.pairwise_corr(df_emp, method='pearson')`으로 모든 변수 쌍의 상관분석을 수행하세요.
유의한 상관(p < 0.05)을 보이는 쌍을 모두 찾으세요.

**(b)** 유의한 상관을 보이는 변수 쌍 중에서, **상관이 인과를 의미하지 않는** 예시를 하나 들고,
왜 인과라고 할 수 없는지 교란변수의 가능성을 포함하여 설명하세요.

> **상관 vs 인과**: 두 변수가 함께 움직여도(상관), 한 변수가 다른 변수를 변화시킨다(인과)고
> 단정할 수 없습니다. 인과를 확인하려면 **실험(RCT, A/B 테스트)** 이 필요합니다.

**(c)** 만약 `경력`과 `만족도` 사이에 유의한 상관이 있다면,
"경력이 많을수록 만족도가 높다"고 결론 내릴 수 있나요?
왜 그런지/아닌지 교란변수(예: 직급, 급여, 업무 환경)의 관점에서 논의하세요.

In [ ]:
print("\n[문제 2-3] 다변량 상관분석과 해석")
print("-" * 40)

# (a) pairwise_corr
print("(a) 모든 변수 쌍의 Pearson 상관분석:")
pearson_result = pg.pairwise_corr(df_emp, method='pearson')
display(pearson_result)

sig_pairs = pearson_result[pearson_result['p-unc'] < 0.05]
print(f"\n    유의한 상관 (p < 0.05):")
for _, row in sig_pairs.iterrows():
    print(f"    {row['X']} <-> {row['Y']}: r={row['r']:.4f}, p={row['p-unc']:.4f}")

# (b) 상관 vs 인과
print(f"\n(b) 상관 ≠ 인과 예시:")
print(f"    변수 쌍: 프로젝트수 <-> 월급여")
print(f"    교란변수 가능성: '경력'이 교란변수로 작용합니다.")
print(f"      - 경력이 많으면 프로젝트를 많이 완료합니다")
print(f"      - 경력이 많으면 월급여도 높습니다")
print(f"      - 따라서 프로젝트수와 월급여의 상관은 경력이라는 제3변수에 의해 발생할 수 있으며,")
print(f"      - 프로젝트를 많이 한다고 급여가 올라간다(인과)고 단정할 수 없습니다.")

# (c) 경력 vs 만족도
r_exp_sat, p_exp_sat = stats.pearsonr(experience, satisfaction)
print(f"\n(c) 경력과 만족도의 관계:")
print(f"    상관 결과: r={r_exp_sat:.4f}, p={p_exp_sat:.4f}")
if p_exp_sat < 0.05:
    print(f"    유의한 상관이 있지만, '경력이 많을수록 만족도가 높다'고 인과 결론을 내릴 수 없습니다.")
else:
    print(f"    유의한 상관이 없으므로 경력과 만족도 사이에 선형 관계가 있다고 보기 어렵습니다.")
print(f"    인과 결론 가능 여부: 불가능합니다 (관찰 연구이므로)")
print(f"    교란변수: 직급, 업무 환경, 팀 분위기, 개인 성향 등이 만족도에 영향을 줄 수 있으며,")
print(f"    경력은 이런 변수들과 함께 변화하기 때문에 인과를 분리하려면 실험(RCT)이 필요합니다.")

---

## 문제 3: 앱 UI 변경 A/B 테스트 (전환율 비교)

모바일 쇼핑 앱에서 결제 화면 UI를 개선하면 **구매 전환율**이 높아지는지 A/B 테스트를 수행합니다.

- **대조군(A)**: 기존 결제 화면
- **실험군(B)**: 개선된 결제 화면 (버튼 크기 확대 + 단계 축소)
- **지표**: 구매 전환율 (결제 완료 비율)
- **현재 전환율**: 8%
- **비즈니스 목표**: 최소 2%p 이상 개선을 탐지하고 싶음 (8% → 10%)

**분석 목표**: 개선된 UI가 기존 대비 전환율을 유의하게 높이는지 검정합니다.

---

### 문제 3-1: 실험 설계 — 표본 크기 산정

A/B 테스트를 실행하기 **전에** 필요한 표본 크기를 산정합니다.

**(a)** 실험 설계 파라미터를 정리하세요.

**(b)** **효과크기(Cohen's h)** 를 계산하세요.

**(c)** **집단당 필요 표본 크기**를 산정하세요.

**(d)** 양측검정과 단측검정 각각에 대해 표본 크기를 구하고, 차이를 비교하세요.
이 실험에서는 어떤 검정이 더 적절한지 근거와 함께 설명하세요.

In [ ]:
print("\n[문제 3] 앱 UI 변경 A/B 테스트")
print("=" * 50)
print("\n[문제 3-1] 실험 설계 — 표본 크기 산정")
print("-" * 40)

baseline_rate = 0.08   # 기존 전환율
mde = 0.02             # 최소 탐지 효과 (2%p)
alpha = 0.05           # 유의수준
power_target = 0.80    # 검정력

# (a) 파라미터 정리
print("(a) 실험 설계 파라미터:")
print(f"    기존 전환율 (p₀): {baseline_rate:.0%}")
print(f"    목표 전환율 (p₁): {baseline_rate + mde:.0%}")
print(f"    MDE: {mde:.0%}p")
print(f"    α: {alpha}")
print(f"    1-β: {power_target}")

# (b) 효과크기 (Cohen's h)
effect_size_h = abs(proportion_effectsize(baseline_rate + mde, baseline_rate))

print(f"\n(b) Cohen's h = {effect_size_h:.4f}")
size_h_label = '매우 작은' if effect_size_h < 0.2 else '작은' if effect_size_h < 0.5 else '중간'
print(f"    해석: {size_h_label} 효과 크기 (0.2/0.5/0.8 기준)")

# (c) 필요 표본 크기
power_analysis = NormalIndPower()
n_two_sided = int(np.ceil(power_analysis.solve_power(
    effect_size=effect_size_h, alpha=alpha, power=power_target, alternative='two-sided')))
n_one_sided = int(np.ceil(power_analysis.solve_power(
    effect_size=effect_size_h, alpha=alpha, power=power_target, alternative='larger')))

print(f"\n(c) 필요 표본 크기:")
print(f"    양측검정: 집단당 {n_two_sided:,}명 (총 {n_two_sided*2:,}명)")
print(f"    단측검정: 집단당 {n_one_sided:,}명 (총 {n_one_sided*2:,}명)")

# (d) 양측 vs 단측 비교 및 선택
print(f"\n(d) 검정 방향 선택:")
print(f"    양측 총 필요: {n_two_sided*2:,}명")
print(f"    단측 총 필요: {n_one_sided*2:,}명")
print(f"    선택: 단측검정 (larger)")
print(f"    근거: UI '개선'의 목적이 전환율을 '높이는 것'이므로 방향이 사전에 명확합니다.")
print(f"    전환율이 오히려 낮아지는 경우는 비즈니스 관점에서 관심 대상이 아닙니다.")
print(f"    따라서 단측검정(larger)이 적절하며, 검정력도 더 높습니다.")

### 문제 3-2: A/B 테스트 실행 및 검정

실험을 진행하여 아래 데이터를 수집했습니다.

**(a)** 각 집단의 전환율을 계산하세요.

**(b)** **z-비율검정**을 수행하세요.

> `proportions_ztest(count, nobs, alternative=...)`

**(c)** **효과크기(Cohen's h)** 를 관측된 전환율로 계산하세요.

**(d)** 각 집단의 **95% 신뢰구간**을 구하세요.

> `proportion_confint(count, nobs, alpha=0.05, method='wilson')`

**(e)** **비율 차이의 95% 신뢰구간**을 구하세요.

> `confint_proportions_2indep(count1, nobs1, count2, nobs2, method='wald')`

**(f)** **상대적 개선율(Relative Lift)** 을 계산하세요.

> 상대적 개선율 = (실험군 전환율 − 대조군 전환율) / 대조군 전환율 × 100

**(g)** 전환율 비교 막대 그래프를 그리세요 (95% 신뢰구간 포함).

In [ ]:
print("\n[문제 3-2] A/B 테스트 실행 및 검정")
print("-" * 40)

np.random.seed(553)
n_per_group = 2600  # 실험에 참여한 인원 (집단당)

# 데이터 생성 (시뮬레이션)
control_conv = np.random.binomial(1, 0.08, n_per_group)    # 대조군: 8%
treatment_conv = np.random.binomial(1, 0.105, n_per_group)  # 실험군: 10.5%

# (a) 전환율 계산
control_rate = control_conv.mean()
treatment_rate = treatment_conv.mean()

print(f"(a) 전환율:")
print(f"    대조군: {control_conv.sum()}/{n_per_group} = {control_rate:.4f} ({control_rate:.2%})")
print(f"    실험군: {treatment_conv.sum()}/{n_per_group} = {treatment_rate:.4f} ({treatment_rate:.2%})")
print(f"    차이: {treatment_rate - control_rate:.4f} ({(treatment_rate - control_rate):.2%}p)")

# (b) z-비율검정 (단측)
count = np.array([treatment_conv.sum(), control_conv.sum()])
nobs = np.array([n_per_group, n_per_group])

z_stat, z_p = proportions_ztest(count, nobs, alternative='larger')

print(f"\n(b) z-비율검정 (단측, alternative='larger'):")
print(f"    z = {z_stat:.4f}")
print(f"    p-value = {z_p:.4f}")
verdict_z = "H₀ 기각 → 실험군의 전환율이 유의하게 높습니다" if z_p <= 0.05 else "H₀ 기각 실패"
print(f"    판정 (α=0.05): {verdict_z}")

# (c) 관측된 효과크기 (Cohen's h)
observed_h = abs(proportion_effectsize(treatment_rate, control_rate))

print(f"\n(c) Cohen's h = {observed_h:.4f}")
obs_h_label = '매우 작은' if observed_h < 0.2 else '작은' if observed_h < 0.5 else '중간'
print(f"    해석: {obs_h_label} 효과 크기입니다")

# (d) 95% 신뢰구간
ci_control = proportion_confint(control_conv.sum(), n_per_group, alpha=0.05, method='wilson')
ci_treatment = proportion_confint(treatment_conv.sum(), n_per_group, alpha=0.05, method='wilson')

print(f"\n(d) 95% 신뢰구간:")
print(f"    대조군: [{ci_control[0]:.4f}, {ci_control[1]:.4f}]")
print(f"    실험군: [{ci_treatment[0]:.4f}, {ci_treatment[1]:.4f}]")

# (e) 비율 차이의 95% 신뢰구간
ci_diff_low, ci_diff_high = confint_proportions_2indep(
    treatment_conv.sum(), n_per_group,
    control_conv.sum(), n_per_group,
    method='wald'
)
diff = treatment_rate - control_rate

print(f"\n(e) 비율 차이의 95% 신뢰구간:")
print(f"    차이 (실험군 − 대조군): {diff:.4f} ({diff:.2%}p)")
print(f"    95% CI: [{ci_diff_low:.4f}, {ci_diff_high:.4f}]")
ci_contains_zero = ci_diff_low <= 0 <= ci_diff_high
print(f"    해석: 신뢰구간이 0을 {'포함합니다 → 차이가 유의하지 않습니다' if ci_contains_zero else '포함하지 않습니다 → 차이가 유의합니다'}")

# (f) 상대적 개선율
relative_lift = (treatment_rate - control_rate) / control_rate * 100

print(f"\n(f) 상대적 개선율: {relative_lift:+.1f}%")

In [ ]:
# (g) 전환율 비교 막대 그래프
fig, ax = plt.subplots(figsize=(8, 6))

labels = ['대조군\n(기존 UI)', '실험군\n(개선 UI)']
rates = [control_rate * 100, treatment_rate * 100]
bar_colors = ['#3B82F6', '#F59E0B']

ci_errors = [
    [control_rate * 100 - ci_control[0] * 100, treatment_rate * 100 - ci_treatment[0] * 100],
    [ci_control[1] * 100 - control_rate * 100, ci_treatment[1] * 100 - treatment_rate * 100]
]

bars = ax.bar(labels, rates, color=bar_colors, alpha=0.8, edgecolor='white',
              yerr=ci_errors, capsize=8)

for bar, rate in zip(bars, rates):
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.3,
            f'{rate:.2f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('전환율 (%)', fontsize=12)
ax.set_title(f'A/B 테스트 — 전환율 비교 (p={z_p:.4f})', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 문제 3-3: 비즈니스 의사결정

**(a)** A/B 테스트 결과를 종합하여, UI 변경을 **전체 사용자에게 적용할지** 의사결정을 내리세요.
아래 항목을 모두 포함하여 보고서 형식으로 작성하세요.

> | 항목 | 내용 |
> |------|------|
> | 통계적 유의성 | p-value와 유의수준 비교 |
> | 효과크기 | Cohen's h와 해석 |
> | 실무적 의미 | 상대적 개선율과 비즈니스 임팩트 |
> | 신뢰구간 | 비율 차이의 신뢰구간과 두 집단의 신뢰구간 겹침 여부 |
> | 최종 권고 | 적용 / 미적용 / 추가 실험 필요 |

**(b)** 만약 p-value는 유의하지만 효과크기가 매우 작다면(예: Cohen's h < 0.1),
어떤 의사결정이 적절한지 설명하세요.

In [ ]:
print("\n[문제 3-3] 비즈니스 의사결정")
print("-" * 40)

# (a) 종합 보고서
print("(a) A/B 테스트 결과 보고서:")
print(f"    [통계적 유의성] p={z_p:.4f} {'<' if z_p <= 0.05 else '>'} 0.05 → {'유의' if z_p <= 0.05 else '비유의'}")
print(f"    [효과크기] Cohen's h = {observed_h:.4f} ({obs_h_label})")
print(f"    [실무적 의미] 상대적 개선율 {relative_lift:+.1f}%")
print(f"    [신뢰구간] 비율 차이 95% CI: [{ci_diff_low:.4f}, {ci_diff_high:.4f}]")
print(f"                대조군 [{ci_control[0]:.4f}, {ci_control[1]:.4f}]")
print(f"                실험군 [{ci_treatment[0]:.4f}, {ci_treatment[1]:.4f}]")
if z_p <= 0.05 and observed_h >= 0.1:
    print(f"    [최종 권고] 적용 권고 — 통계적으로 유의하고 효과크기도 실무적 의미가 있습니다.")
elif z_p <= 0.05:
    print(f"    [최종 권고] 조건부 적용 — 유의하지만 효과가 작으므로 구현 비용과 비교하여 판단합니다.")
else:
    print(f"    [최종 권고] 미적용 — 통계적으로 유의하지 않습니다. 추가 실험이 필요합니다.")

# (b) 유의하지만 효과 작은 경우
print(f"\n(b) p 유의 + 효과크기 작은 경우의 의사결정:")
print(f"    Cohen's h < 0.1이면 통계적으로 유의하더라도 실무적 의미가 매우 작습니다.")
print(f"    표본 크기가 매우 크면 아주 작은 차이도 유의하게 나올 수 있으므로(과검정력),")
print(f"    효과크기와 비즈니스 임팩트를 함께 고려해야 합니다.")
print(f"    이 경우 UI 변경의 개발/유지보수 비용 대비 효과가 미미하므로,")
print(f"    적용하지 않거나 다른 개선안을 시도하는 것이 적절합니다.")

---

## 문제 4: 학습 앱 A/B 테스트 (연속형 지표 — 학습 시간 비교)

온라인 학습 앱에서 새로운 **게이미피케이션 기능**(배지, 레벨업 시스템)을 도입하면
일일 학습 시간이 늘어나는지 A/B 테스트를 진행합니다.

- **대조군(A)**: 기존 앱 (게이미피케이션 없음)
- **실험군(B)**: 게이미피케이션 도입 버전
- **지표**: 일일 학습 시간 (분)

**분석 목표**: 게이미피케이션 도입이 일일 학습 시간을 유의하게 증가시키는지 검정합니다.

**주어진 데이터:**

In [ ]:
print("\n[문제 4] 학습 앱 A/B 테스트 — 학습 시간 비교")
print("=" * 50)

np.random.seed(554)
n_ab = 300

# 학습 시간 데이터 (지수분포 — 우측 꼬리가 긴 비대칭 분포)
time_control = np.round(np.random.exponential(scale=25, size=n_ab), 1)    # 대조군: 평균 25분
time_treatment = np.round(np.random.exponential(scale=30, size=n_ab), 1)  # 실험군: 평균 30분

print(f"  대조군 (n={n_ab}): 평균={time_control.mean():.1f}분, 중앙값={np.median(time_control):.1f}분, SD={time_control.std(ddof=1):.1f}분")
print(f"  실험군 (n={n_ab}): 평균={time_treatment.mean():.1f}분, 중앙값={np.median(time_treatment):.1f}분, SD={time_treatment.std(ddof=1):.1f}분")

In [ ]:
# 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].hist(time_control, bins=25, alpha=0.5, color='#3B82F6', edgecolor='white',
             label=f'대조군 (평균={time_control.mean():.1f}분)', density=True)
axes[0].hist(time_treatment, bins=25, alpha=0.5, color='#F59E0B', edgecolor='white',
             label=f'실험군 (평균={time_treatment.mean():.1f}분)', density=True)
axes[0].axvline(time_control.mean(), color='#3B82F6', linestyle='--', linewidth=2)
axes[0].axvline(time_treatment.mean(), color='#F59E0B', linestyle='--', linewidth=2)
axes[0].set_xlabel('학습 시간 (분)', fontsize=12)
axes[0].set_ylabel('밀도', fontsize=12)
axes[0].set_title('학습 시간 분포 비교', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=9)
axes[0].grid(alpha=0.3)

bp = axes[1].boxplot([time_control, time_treatment],
                      labels=['대조군', '실험군'],
                      patch_artist=True, widths=0.5)
bp['boxes'][0].set_facecolor('#3B82F6')
bp['boxes'][0].set_alpha(0.6)
bp['boxes'][1].set_facecolor('#F59E0B')
bp['boxes'][1].set_alpha(0.6)
axes[1].set_ylabel('학습 시간 (분)', fontsize=12)
axes[1].set_title('집단별 학습 시간 비교', fontsize=13, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 문제 4-1: 정규성 확인과 검정 방법 결정

**(a)** 각 집단의 **정규성 검정**(Shapiro-Wilk)을 수행하세요.

**(b)** 분포의 **왜도(skewness)** 를 계산하세요.

> `from scipy.stats import skew` → `skew(data)`

**(c)** 이 데이터에 t-검정을 적용할 수 있는지 판단하세요.
**중심극한정리(CLT)** 의 관점에서, 표본 크기(n=300)와 왜도를 함께 고려하여 설명하세요.

> | \|skew\| | 분류 | CLT 실무 기준 |
> |:--------:|:---:|:-------------|
> | < 0.5 | 거의 대칭 | n ≥ 15~20이면 충분 |
> | 0.5 ~ 1.0 | 중간 비대칭 | n ≥ 30 권장 |
> | > 1.0 | 강한 비대칭 | n ≥ 100+ 또는 비모수 고려 |

**(d)** 이 데이터에 사용할 **검정 방법**을 결정하세요.
모수 검정(Welch's t)과 비모수 검정(Mann-Whitney U) 중 어떤 것이 적절한지,
또는 **둘 다 수행하여 비교**하는 것이 좋은지 근거를 제시하세요.

In [ ]:
print("\n[문제 4-1] 정규성 확인과 검정 방법 결정")
print("-" * 40)

# (a) 정규성 검정
print("(a) Shapiro-Wilk 정규성 검정:")
for name, data in [('대조군', time_control), ('실험군', time_treatment)]:
    stat_sw, p_sw = stats.shapiro(data)
    result = "정규성 유지" if p_sw > 0.05 else "정규성 위반"
    print(f"    {name}: W={stat_sw:.4f}, p={p_sw:.4f} → {result}")

# (b) 왜도
skew_control = skew(time_control)
skew_treatment = skew(time_treatment)

print(f"\n(b) 왜도:")
print(f"    대조군: {skew_control:.4f}")
print(f"    실험군: {skew_treatment:.4f}")

# (c) CLT 판단
print(f"\n(c) CLT 관점의 판단:")
print(f"    표본 크기: n={n_ab}")
for name, sk in [('대조군', skew_control), ('실험군', skew_treatment)]:
    if abs(sk) < 0.5:
        cat = "거의 대칭 → n >= 15~20이면 충분"
    elif abs(sk) < 1.0:
        cat = "중간 비대칭 → n >= 30 권장"
    else:
        cat = "강한 비대칭 → n >= 100+ 또는 비모수 고려"
    print(f"    {name}: |skew|={abs(sk):.2f} → {cat}")
print(f"    판단: 왜도가 1.0 이상(강한 비대칭)이지만 n={n_ab}으로 충분히 크므로")
print(f"    CLT에 의해 표본평균의 분포는 정규분포에 근사합니다.")
print(f"    따라서 t-검정을 사용할 수 있습니다.")

# (d) 검정 방법 결정
print(f"\n(d) 검정 방법 결정:")
print(f"    선택: 둘 다 수행하여 비교 (Welch's t-검정 + Mann-Whitney U)")
print(f"    근거: 정규성이 위반되었지만 대표본(n={n_ab})이므로 CLT에 의해 t-검정 사용 가능합니다.")
print(f"    비모수 검정(Mann-Whitney U)도 함께 수행하여 결과의 일관성을 확인합니다.")
print(f"    두 검정의 결론이 일치하면 결과에 대한 신뢰도가 높아집니다.")

### 문제 4-2: 검정 수행 및 효과크기

**(a)** **Welch's t-검정**을 수행하세요 (pingouin의 `pg.ttest()` 사용).

> `pg.ttest(x, y, alternative='two-sided')` → T, p-val, cohen-d, CI95%, BF10, power 제공

**(b)** **Mann-Whitney U 검정**을 수행하세요 (pingouin의 `pg.mwu()` 사용).

> `pg.mwu(x, y, alternative='two-sided')` → U-val, p-val, RBC, CLES 제공

**(c)** 두 검정의 결과를 비교하세요. 결론이 같은가요, 다른가요?
왜 같은/다른 결론이 나오는지 데이터의 특성(분포, 표본 크기)과 연결하여 설명하세요.

**(d)** 검정 결과를 종합하여, 게이미피케이션 도입의 효과에 대한 최종 결론을 내리세요.
통계적 유의성, 효과크기, 상대적 개선율을 모두 포함하세요.

In [ ]:
print("\n[문제 4-2] 검정 수행 및 효과크기")
print("-" * 40)

# (a) Welch's t-검정
print("(a) Welch's t-검정:")
ttest_result = pg.ttest(time_treatment, time_control, alternative='two-sided')
display(ttest_result)

# (b) Mann-Whitney U 검정
print(f"\n(b) Mann-Whitney U 검정:")
mwu_result = pg.mwu(time_treatment, time_control, alternative='two-sided')
display(mwu_result)

# (c) 두 검정 비교
t_p_val = ttest_result['p-val'].values[0]
t_cohen_d = ttest_result['cohen-d'].values[0]
mwu_p_val = mwu_result['p-val'].values[0]
mwu_rbc = mwu_result['RBC'].values[0]

print(f"\n(c) 검정 결과 비교:")
print(f"    t-검정: p = {t_p_val:.4f}, Cohen's d = {t_cohen_d:.4f}")
print(f"    Mann-Whitney: p = {mwu_p_val:.4f}, RBC = {mwu_rbc:.4f}")

t_verdict = "유의" if t_p_val < 0.05 else "비유의"
mwu_verdict = "유의" if mwu_p_val < 0.05 else "비유의"

if t_verdict == mwu_verdict:
    print(f"    결론 일치 여부: 일치 (둘 다 {t_verdict})")
    print(f"    이유: 표본 크기가 충분히 커서(n={n_ab}) CLT가 잘 작동하고,")
    print(f"    두 검정 모두 동일한 결론에 도달했습니다.")
else:
    print(f"    결론 일치 여부: 불일치 (t: {t_verdict}, MWU: {mwu_verdict})")
    print(f"    이유: 지수분포의 강한 비대칭과 이상치가 두 검정에 다르게 영향을 미칩니다.")

# (d) 최종 결론
lift_pct = (time_treatment.mean() - time_control.mean()) / time_control.mean() * 100
d_label = '매우 작은' if abs(t_cohen_d) < 0.2 else '작은' if abs(t_cohen_d) < 0.5 else '중간' if abs(t_cohen_d) < 0.8 else '큰'

print(f"\n(d) 최종 결론:")
print(f"    [통계적 유의성] Welch's t p={t_p_val:.4f}, Mann-Whitney p={mwu_p_val:.4f}")
print(f"    [효과크기] Cohen's d = {t_cohen_d:.4f} ({d_label} 효과)")
print(f"    [상대적 개선율] {lift_pct:+.1f}%")
if t_p_val < 0.05:
    print(f"    [결론] 게이미피케이션 도입은 일일 학습 시간을 유의하게 증가시켰습니다.")
    print(f"    평균 학습 시간이 {time_control.mean():.1f}분에서 {time_treatment.mean():.1f}분으로")
    print(f"    약 {time_treatment.mean() - time_control.mean():.1f}분({lift_pct:+.1f}%) 증가했습니다.")
else:
    print(f"    [결론] 게이미피케이션 도입의 효과가 통계적으로 유의하지 않습니다.")
    print(f"    추가 실험이나 다른 개선안을 검토할 필요가 있습니다.")

---
### 정답 요약

In [ ]:
print("\n" + "=" * 60)
print("정답 요약")
print("=" * 60)

print(f"\n[문제 1] ANOVA — 공장별 배터리 수명")
print(f"  가설: H₀: μ_A = μ_B = μ_C / H₁: 적어도 하나 다름")
print(f"  정규성: 모든 집단 통과 / 등분산: Levene p={lev_p:.4f} (충족)")
print(f"  One-way ANOVA: F={f_stat:.2f}, p={f_p:.6f}")
print(f"  η²={eta_sq:.4f}, ω²={omega_sq:.4f}")
print(f"  Tukey HSD: 3쌍 모두 유의한 차이")

print(f"\n[문제 2] 상관분석 — 직원 데이터")
print(f"  Pearson r(경력-급여)={r_pearson:.4f}, p={p_pearson:.4f}")
print(f"  Spearman ρ(경력-급여)={r_spearman:.4f}, p={p_spearman:.4f}")
print(f"  r²={r_squared:.4f}")

print(f"\n[문제 3] A/B 테스트 — 전환율")
print(f"  설계: Cohen's h={effect_size_h:.4f}")
print(f"  필요 표본: 양측 {n_two_sided:,}명, 단측 {n_one_sided:,}명")
print(f"  z-비율검정: z={z_stat:.4f}, p={z_p:.4f}")
print(f"  Cohen's h(관측)={observed_h:.4f}")
print(f"  비율 차이 95% CI: [{ci_diff_low:.4f}, {ci_diff_high:.4f}]")
print(f"  상대적 개선율: {relative_lift:+.1f}%")

print(f"\n[문제 4] 연속형 A/B 테스트 — 학습 시간")
print(f"  왜도: 대조군={skew_control:.2f}, 실험군={skew_treatment:.2f}")
print(f"  Welch's t: p={t_p_val:.4f}, Cohen's d={t_cohen_d:.4f}")
print(f"  Mann-Whitney: p={mwu_p_val:.4f}, RBC={mwu_rbc:.4f}")
print(f"  상대적 개선율: {lift_pct:+.1f}%")